In [1]:
# Практическое задание
#
# Создать объект DataFrame из текста с частотами похожих слов
#
# Добавленное условие -- исключить числа

In [2]:
import re
import pandas as pd

In [3]:
FILEPATH = 'text.txt'

In [4]:
def from_file(filepath=FILEPATH, encod="cp1251"):
    """ Read text from file """
    # для действительно больших текстов такой спопоб не подходит
    with open(filepath or FILEPATH, "r", encoding=encod) as file_obj:
        return file_obj.read()

In [5]:
def distance(a, b):
    "Calculates the Levenshtein distance between a and b."
    n, m = len(a), len(b)
    if n > m:
        # Make sure n <= m, to use O(min(n,m)) space
        a, b = b, a
        n, m = m, n
    i, j = 0, 0
    current_row = range(n+1) # Keep current and previous row, not entire matrix
    for i in range(1, m+1):
        previous_row, current_row = current_row, [i]+[0]*n
        for j in range(1, n+1):
            add, delete, change = previous_row[j]+1, current_row[j-1]+1, previous_row[j-1]
            if a[j-1] != b[i-1]:
                change += 1
            current_row[j] = min(add, delete, change)

    return current_row[n]

In [6]:
def data_preparation(input_data):
    """prepare data for next work"""
    
    data_new = input_data[:]

    #приведение к нижнему регистру и разбивка на слова
    data_new = pd.DataFrame(re.split(r"\W+", data_new.lower())[:-1], columns=["word"])

    # группировка по словам, одновременно формируется список уникальных слов
    data_new["cnt"] = 1
    data_new = data_new.groupby(["word"], as_index=True).sum()

    # удаление чисел
    pattern = re.compile(r"^\d+$")
    idx = (el for el in data_new.index if not pattern.match(el))
    data_new = data_new.loc[idx]
    
    return data_new

In [7]:
def create_dict(input_data):
    """Create dictionary with similar words"""
    # возвращает словарь с похожими словами и объект Serias с ключевым словом для каждого слова в индексе
    data_new = input_data.copy()
    data_new["is_selected"] = ""
    #  похожие слова ищем среди слов с 3-мя и более буквами
    #  среди 1 и 2-х буквенных слов похожих не ожидается
    idx = [el for el in data_new.index if len(el)>2]
    # df -- объект Serias, индексы -- перечень слов с 3 и более буквами, 
    # значения -- "", если слово еще не включено уже в какую-либо группу похожих слов,
    # или ключевое слово группы похожих слов
    df = data_new["is_selected"].loc[idx]
    dict_synonym = {}

    for i, idx_i in enumerate(df.index[:-1]):
        if df[idx_i]:  # or i>100:   # для уменьшения времени работы для тестирования  
            continue
        else:
            dict_synonym[idx_i] = [idx_i]  # еще не включенное слово становиться ключем для группы
            df[idx_i] = idx_i
            for idx_j in df.index[i+1:]:  # среди оставшихся слов, ищем слова, которые можно включить в группу
                if not df[idx_j]:
                    min_len = min(len(idx_i), len(idx_j))
                    dist = distance(idx_i, idx_j)
                    if (min_len<=5 and dist<=1) or (6<=min_len<=8 and dist<=2) or (9<=min_len and dist<=3):
                        df[idx_j]=idx_i
                        dict_synonym[idx_i].append(idx_j)
    
    # добавлено, так как последний элемент может быть пропущен в цикле
    if not df.iloc[-1]: 
        dict_synonym[df.index[-1]] = [df.index[-1]]
        df[df.index[-1]] = df.index[-1]
    
    # добавляем оставшиеся 1 и 2 буквенные слова
    apd = data_new["is_selected"].loc[set(data_new.index) - set(idx)]
    apd[apd.index] = apd.index
    df = df.append(apd)
    dict_apd = apd.to_dict()
    dict_apd = {el:[dict_apd[el]] for el in dict_apd}
    dict_synonym.update(dict_apd)
    
    return dict_synonym, df

In [8]:
data = from_file()
data = data_preparation(data)

In [9]:
%%time
dict_synonym, df = create_dict(data)

# Wall time: 5min 11s

Wall time: 5min 8s


In [10]:
words_0 = data.cnt.sum()  # для последующей проверки

In [11]:
data_2 = data.copy()
data_2["key_word"] = df
data_2 = data_2.groupby(data_2.key_word).sum()
data_2 = data_2.transpose()

In [12]:
words_1 = data_2.loc["cnt"].sum()
if words_0==words_1:
    print("No lost words")
else:
    print("Missed a few words")

No lost words


In [13]:
# print(dict_synonym)
print(data_2)
#data_2.to_csv("result.csv", encoding="cp1251")

key_word  2е  30х30х30  3е  c  it  y  ya  z    а  аб    ...     ярослав  ясно  \
cnt        1         1   1  2   2  1   1  1  108   1    ...          11     1   

key_word  і  іграе  ідэі  ў  ўлады  ўсе  ўсеагульнае  ўяўляюць  
cnt       3      1     1  3      1    1            1         1  

[1 rows x 2874 columns]
